## Clinical disease data Analysis

### Problem Statement
Your boss comes to you Monday morning and says “I figured out our next step; we are going to pivot from an online craft store and become a data center for genetic disease information! I found **ClinVar** which is a repository that contains expert curated data, and it is free for the taking. This is a gold mine! Look at the file and tell me what gene and mutation combinations are classified as dangerous.”

Make sure that you only give your boss the dangerous mutations and include:

1) Gene name

2) Mutation ID number

3) Mutation Position (chromosome & position)

4) Mutation value (reference & alternate bases)

5) Clinical severity 

6) Disease that is implicated

**Requirements**

1) The deliverables are the final result as a dataframe with a short discussion of any specifics. (that is, what data you would present to your boss with the explanation of your results)

2) Limit your output to the first 100 harmful mutations and tell your boss how many total harmful mutations were found in the file

3) Use the "clinvar_final.txt" at this link: https://drive.google.com/file/d/1Zps0YssoJbZHrn6iLte2RDLlgruhAX1s/view?usp=sharing This file is modified to be not exactly the same as 'standard' .vcf file. **This is a large file so won't be uploaded into github repo!**

4) Replace missing values in the dataframe with: 'Not_Given'. Print or display this for the column `CLNSIG` by using value_counts().

5) State in your answer how you define harmful mutations

**6) Do your best on getting to above requirements and submit whatever you do before the deadline. If your work is incomplete be sure to describe the blockers that got in your way and how you might get past them (if given more time).**

7) You can use as many code blocks as you need. Please clean-up your code and make it readable for the graders!

**Hints** 
* We do not expect you to have any medical knowledge to solve this problem; look at the data, read the documentation provided, and write down your assumptions!


* Map out which fields you want to extract: Are they in the same place every time? What strategy will you use to robustly extract and filter your data of interest? How do you plan to handle missing data?

* A good way to start is to print out each line, then practice parsing them to see if you can recover the fields of interest

* A starting solution for parsing .vcfs can be found here: https://gist.github.com/dceoy/99d976a2c01e7f0ba1c813778f9db744 This solution does **NOT** work due to the changes to standard vcf file. 

* Filter out junk and lines with no mutation data. Just focus on the data your need to deliver to your boss. 

* Pandas and NumPy parsers correctly recognize the end of each line in in the ClinVar file.

* The unit of observation of this dataset is one row per mutation.

* This is similar to a task that one of us tackled at work. You can answer the question with the information provided below or using the (partial) data dictionary file at this link: https://drive.google.com/file/d/1lx9yHdlcqmU_OlHiTUXKC_LQDqYBypH_/view?usp=sharing. Put together a sensible plan, implement a solid parsing strategy, and document and justify the decisions that you made.

### VCF file description (Summarized from version 4.1)

```
* The VCF specification:

VCF is a text file format which contains meta-information lines, a header line, and then data lines each containing information about a position in the genome. The format also can contain genotype information on samples for each position.

* Fixed fields:

There are 8 fixed fields per record. All data lines are **tab-delimited**. In all cases, missing values are specified with a dot (‘.’). 

1. CHROM - chromosome number
2. POS - position DNA nuceleotide count (bases) along the chromosome
3. ID - The unique identifier for each mutation
4. REF - reference base(s)
5. ALT - alternate base(s)
6. FILTER - filter status
7. QUAL - quality
8. INFO - a semicolon-separated series of keys with values in the format: <key>=<data>

```
### Applicable INFO field specifications

```
GENEINFO = <Gene name>
CLNSIG =  <Clinical significance>
CLNDN = <Disease name>
```

### Sample ClinVar data (vcf file format - not exactly the same as the file to download!)

```
##fileformat=VCFv4.1
##fileDate=2019-03-19
##source=ClinVar
##reference=GRCh38							
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO
1	949523	rs786201005	C	T	.	.	GENEINFO=ISG15;CLNSIG=5
1	949696	rs672601345	C	CG	.	.	GENEINFO=ISG15;CLNSIG=5;CLNDN=Cancer
1	949739	rs672601312	G	T	.	.	GENEINFO=ISG15;CLNDBN=Cancer
1	955597	rs115173026	G	T	.	.	GENEINFO=AGRN;CLNSIG=2; CLNDN=Cancer
1	955619	rs201073369	G	C	.	.	GENEINFO=AGG;CLNDN=Heart_dis 
1	957640	rs6657048	C	T	.	.	GENEINFO=AGG;CLNSIG=3;CLNDN=Heart_dis 
1	976059	rs544749044	C	T	.	.	GENEINFO=AGG;CLNSIG=0;CLNDN=Heart_dis 
```

In [1]:
# Reading clinvar_final.txt and loading in dataframe 'df'
import pandas as pd
import io
pd.options.display.max_columns=20
with open('clinvar_final.txt', 'r') as f:
    lines = [l for l in f if not l.startswith('#')]

df = pd.read_csv(io.StringIO(''.join(lines)), dtype={'CHROM': str, 'POS': str, 'ID': int, 'REF': str, 'ALT': str, 'QUAL': str, 'FILTER': str, 'INFO': str},
        sep='\t')

df
    

,CHROM,POS,ID,REF,ALT,FILTER,QUAL,INFO
0,1,1014O42,475283,G,A,.,.,AF_ESP=0.00546;AF_EXAC=0.00165;AF_TGP=0.00619;...
1,1,1O14122,542074,C,T,.,.,AF_ESP=0.00015;AF_EXAC=0.00010;ALLELEID=514926...
2,1,1014143,183381,C,T,.,.,"ALLELEID=181485;CLNDISDB=MedGen:C4015293,OMIM:..."
3,1,1014179,542075,C,T,.,.,"ALLELEID=514896;CLNDISDB=MedGen:C4015293,OMIM:..."
4,1,1014217,475278,C,T,.,.,AF_ESP=0.00515;AF_EXAC=0.00831;AF_TGP=0.00339;...
...,...,...,...,...,...,...,...,...
102316,3,179210507,403908,A,G,.,.,"ALLELEID=393412;CLNDISDB=MedGen:C0018553,Orpha..."
102317,3,179210511,526648,T,C,.,.,"ALLELEID=519163;CLNDISDB=MedGen:C0018553,Orpha..."
102318,3,179210515,526640,A,C,.,.,AF_EXAC=0.00002;ALLELEID=519178;CLNDISDB=MedGe...
102319,3,179210516,246681,A,G,.,.,AF_EXAC=0.00001;ALLELEID=245287;CLNDISDB=MedGe...


# Pseudocode

1) Read clinvar_final.txt and load in dataframe 'df'

2) Parse 'INFO' column from dataframe 'df' to extract applicable info field specifications : 'GENEINFO', 'CLNSIG', 'CLNDN' 

3) Create required column 'Mutation Position' (Chromosome & Position) and 'Mutation Value' (reference & alternate bases) by joining the corresponding columns from original dataframe 'df'

4) Create another subset of dataframe 'df_subset' by keeping only 6 columns as per the requirement

5) Find missing values from dataframe 'df_subset' and replace missing values in dataframe 'df_subset' with 'Not_Given' for column'CLNSIG'

7) Create final dataframe 'df_final' by renaming columns as per the requirement

8) Display missing values for column 'Clinical severity'/'CLNSIG' replaced by 'Not_Given' using value_counts()

9) Create list of harmful mutations based on values for 'Clinical severity'/'CLNSIG'. Then, create a dataframe 'df_harmful_mutations' based on just harmful mutations. Count and display them using shape. This will give the total number of harmful mutations found in the file.

10) Limit output to the first 100 harmful mutations by using head(100).